In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
sys.path.append(os.path.join(os.getcwd(),'../../../..'))
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/input_configuration.toml'))
summary_config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/summary_configuration.toml'))

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
output_path = os.path.join(summary_config['sc_run_path'], summary_config["output_folder"])
survey_path = os.path.join(summary_config['sc_run_path'], summary_config["survey_folder"])
output_dir = os.path.join(output_path,'RTP\mode_share')

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
pd.options.display.float_format = '{:0,.1%}'.format

mode_share_county = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_county.csv'))
df_person = pd.read_csv(os.path.join(output_path,'agg\dash\person_geog.csv'))

### Region

In [5]:
result_df = pd.DataFrame()

df_trip = mode_share_county.copy()
df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'
for purp in ['All Trips','Work','Non-Work']:
    if purp != 'All Trips':
        df = df_trip[df_trip['trip_type'] == purp]
    else:
        df = df_trip.copy()
    _df = df.groupby('mode').sum()[['trexpfac']]/df['trexpfac'].sum()
    _df['purp'] = purp
    #result_df = result_df.append(_df)
    result_df = pd.concat([result_df, _df])

_df = result_df.pivot_table(columns='purp', index='mode', values='trexpfac', aggfunc='sum')
_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_region.csv')); incr+=1
_df = _df.reset_index()
_df.rename(columns={'mode': 'Mode'}, inplace=True)
_df = _df.rename_axis(None, axis=1)
HTML(_df.to_html(index=False))

Mode,All Trips,Non-Work,Work
Bike,1.5%,1.5%,1.4%
HOV2,21.4%,23.2%,9.4%
HOV3+,14.1%,15.4%,5.9%
SOV,44.5%,40.3%,71.5%
School Bus,1.6%,1.9%,0.0%
TNC,0.3%,0.3%,0.6%
Transit,2.2%,2.2%,2.3%
Walk,14.4%,15.3%,8.9%


### County

In [6]:
def calculate_mode_share(df, geography, purp=None):
    
    df.loc[df['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
    df.loc[df['dpurp'] == 'Work', 'trip_type'] = 'Work'
    
    if purp:
        df = df[df['trip_type'] == purp]
    
    _df = df.pivot_table(index='mode', columns=geography, values='trexpfac', aggfunc='sum')
    
    for col in _df:
        _df[col] = _df[col]/_df[col].sum()
    return _df.T

In [7]:
df = mode_share_county.copy()
df = calculate_mode_share(df, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_county.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_county': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
King,1.6%,20.3%,13.2%,43.3%,1.5%,0.4%,3.3%,16.4%
Kitsap,1.2%,22.3%,14.3%,45.0%,1.5%,0.2%,1.3%,14.2%
Outside Region,NaN,27.3%,27.3%,45.5%,NaN,NaN,NaN,NaN
Pierce,1.4%,23.1%,15.6%,44.9%,1.9%,0.2%,0.8%,12.1%
Snohomish,1.3%,22.4%,15.3%,47.0%,1.7%,0.2%,0.8%,11.3%


In [8]:
df = mode_share_county.copy()
df = calculate_mode_share(df, 'hh_county', 'Non-Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_county_nonwork.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_county': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
King,1.6%,22.1%,14.4%,39.2%,1.7%,0.3%,3.3%,17.2%
Kitsap,1.3%,23.9%,15.4%,41.5%,1.6%,0.2%,0.9%,15.1%
Outside Region,NaN,28.6%,28.6%,42.9%,NaN,NaN,NaN,NaN
Pierce,1.4%,24.9%,16.8%,40.8%,2.2%,0.2%,0.7%,13.0%
Snohomish,1.3%,24.3%,16.6%,42.5%,1.9%,0.2%,0.8%,12.3%


In [9]:
df = mode_share_county.copy()
df = calculate_mode_share(df, 'hh_county', 'Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_county_work.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_county': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
King,1.7%,9.1%,5.6%,68.4%,0.0%,0.8%,3.0%,11.5%
Kitsap,1.0%,9.3%,5.8%,72.1%,0.0%,0.2%,4.4%,7.2%
Outside Region,NaN,NaN,NaN,100.0%,NaN,NaN,NaN,NaN
Pierce,1.1%,9.9%,6.2%,75.9%,0.0%,0.3%,1.3%,5.3%
Snohomish,1.0%,9.9%,6.4%,76.9%,0.0%,0.3%,0.6%,4.9%


## Regional Growth Center

In [10]:
mode_share_rgc = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_rgc.csv'))
df = mode_share_rgc.copy()

# Create results that sum up trips for households in all RGCs
_df = df[df['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
_df['hh_rgc'] = 'All RGCs'
df = pd.concat([df, _df])
df = calculate_mode_share(df, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rgc.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rgc': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
All RGCs,3.0%,10.8%,5.2%,29.0%,0.5%,1.5%,6.5%,43.4%
Auburn,2.3%,19.3%,9.9%,38.3%,1.0%,0.5%,3.4%,25.5%
Bellevue,2.5%,11.6%,5.1%,31.6%,0.4%,0.8%,3.4%,44.6%
Bothell Canyon Park,1.5%,21.3%,11.0%,49.6%,1.4%,0.4%,0.9%,14.0%
Bremerton,2.5%,15.0%,7.1%,34.7%,0.4%,0.5%,2.6%,37.2%
Burien,1.8%,17.7%,9.5%,44.1%,0.7%,0.5%,3.4%,22.3%
Everett,1.8%,13.5%,6.9%,38.8%,0.7%,0.5%,2.1%,35.7%
Federal Way,2.0%,19.6%,14.3%,37.2%,2.0%,0.1%,2.1%,22.8%
Greater Downtown Kirkland,1.8%,17.7%,8.4%,46.7%,0.8%,0.4%,4.0%,20.3%
Kent,2.4%,18.8%,12.7%,35.4%,1.3%,0.7%,4.2%,24.5%


In [11]:
df

,Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
0,All RGCs,3.0%,10.8%,5.2%,29.0%,0.5%,1.5%,6.5%,43.4%
1,Auburn,2.3%,19.3%,9.9%,38.3%,1.0%,0.5%,3.4%,25.5%
2,Bellevue,2.5%,11.6%,5.1%,31.6%,0.4%,0.8%,3.4%,44.6%
3,Bothell Canyon Park,1.5%,21.3%,11.0%,49.6%,1.4%,0.4%,0.9%,14.0%
4,Bremerton,2.5%,15.0%,7.1%,34.7%,0.4%,0.5%,2.6%,37.2%
5,Burien,1.8%,17.7%,9.5%,44.1%,0.7%,0.5%,3.4%,22.3%
6,Everett,1.8%,13.5%,6.9%,38.8%,0.7%,0.5%,2.1%,35.7%
7,Federal Way,2.0%,19.6%,14.3%,37.2%,2.0%,0.1%,2.1%,22.8%
8,Greater Downtown Kirkland,1.8%,17.7%,8.4%,46.7%,0.8%,0.4%,4.0%,20.3%
9,Kent,2.4%,18.8%,12.7%,35.4%,1.3%,0.7%,4.2%,24.5%


In [12]:
df = mode_share_rgc.copy()

# Create results that sum up trips for households in all RGCs
_df = df[df['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
_df['hh_rgc'] = 'All RGCs'
df = pd.concat([df, _df])

df = calculate_mode_share(df, 'hh_rgc', 'Non-Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rgc_nonwork.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rgc': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
All RGCs,3.0%,11.9%,5.8%,26.1%,0.6%,1.2%,6.9%,44.4%
Auburn,2.2%,20.5%,10.6%,35.2%,1.1%,0.5%,3.0%,27.0%
Bellevue,2.4%,12.8%,5.5%,28.1%,0.5%,0.7%,3.7%,46.3%
Bothell Canyon Park,1.4%,23.5%,12.2%,45.1%,1.6%,0.4%,0.7%,15.2%
Bremerton,2.4%,16.1%,7.6%,31.4%,0.5%,0.5%,1.8%,39.8%
Burien,1.8%,18.9%,10.0%,39.4%,0.8%,0.4%,3.6%,25.0%
Everett,1.6%,14.6%,7.6%,34.4%,0.8%,0.5%,2.3%,38.3%
Federal Way,2.0%,20.9%,15.3%,33.3%,2.2%,0.1%,2.2%,24.0%
Greater Downtown Kirkland,1.8%,19.3%,9.1%,42.2%,0.9%,0.4%,4.0%,22.2%
Kent,2.2%,20.0%,13.6%,31.9%,1.5%,0.7%,4.1%,26.1%


In [13]:
df = mode_share_rgc.copy()

# Create results that sum up trips for households in all RGCs
_df = df[df['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
_df['hh_rgc'] = 'All RGCs'
df = pd.concat([df,_df])

df = calculate_mode_share(df, 'hh_rgc', 'Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rgc_work.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rgc': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
All RGCs,3.3%,5.3%,2.5%,43.0%,NaN,2.7%,4.6%,38.6%
Auburn,2.9%,9.1%,4.0%,63.4%,NaN,0.7%,6.5%,13.3%
Bellevue,2.8%,5.8%,2.8%,49.5%,NaN,1.4%,1.9%,35.7%
Bothell Canyon Park,2.1%,8.5%,3.7%,76.3%,NaN,0.3%,1.9%,7.2%
Bremerton,3.3%,7.9%,3.7%,57.6%,NaN,0.5%,7.7%,19.4%
Burien,1.9%,10.2%,6.2%,72.3%,NaN,0.7%,2.1%,6.5%
Everett,2.4%,6.7%,2.8%,67.7%,NaN,0.7%,0.9%,18.9%
Federal Way,2.0%,8.3%,6.0%,70.2%,NaN,NaN,1.2%,12.3%
Greater Downtown Kirkland,1.6%,8.0%,3.8%,73.7%,NaN,0.5%,4.0%,8.5%
Kent,4.1%,9.1%,6.5%,62.6%,NaN,0.3%,5.1%,12.3%


## Regional Geography

In [14]:
mode_share_rg = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_rg.csv'))
df = mode_share_rg.copy()

df = calculate_mode_share(df, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rg.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rg_proposed': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
CitiesTowns,1.2%,22.8%,16.6%,45.0%,2.1%,0.2%,0.6%,11.6%
Core,1.4%,22.2%,14.9%,45.6%,1.7%,0.2%,1.8%,12.1%
HCT,1.3%,22.6%,15.3%,46.2%,1.8%,0.2%,1.3%,11.2%
Metro,2.1%,17.8%,10.7%,40.0%,1.2%,0.6%,4.4%,23.2%
UU,1.2%,24.2%,17.7%,44.9%,2.1%,0.2%,0.5%,9.3%


In [15]:
df = mode_share_rg.copy()
df = calculate_mode_share(df, 'hh_rg_proposed', 'Non-Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rg_nonwork.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rg_proposed': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
CitiesTowns,1.2%,24.5%,18.0%,40.6%,2.4%,0.2%,0.5%,12.6%
Core,1.4%,24.1%,16.1%,41.3%,2.0%,0.2%,1.8%,13.0%
HCT,1.3%,24.5%,16.6%,41.9%,2.1%,0.2%,1.3%,12.2%
Metro,2.0%,19.5%,11.7%,36.0%,1.4%,0.5%,4.5%,24.4%
UU,1.3%,26.1%,19.2%,40.5%,2.4%,0.1%,0.4%,10.0%


In [16]:
df = mode_share_rg.copy()
df = calculate_mode_share(df, 'hh_rg_proposed', 'Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rg_work.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rg_proposed': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
CitiesTowns,0.7%,10.3%,6.9%,76.3%,0.0%,0.2%,0.9%,4.7%
Core,1.2%,10.0%,6.4%,74.3%,0.0%,0.3%,2.0%,5.8%
HCT,1.0%,10.0%,6.4%,75.7%,0.0%,0.3%,1.7%,4.9%
Metro,2.3%,8.3%,4.7%,63.2%,0.0%,1.1%,3.8%,16.6%
UU,0.6%,10.3%,6.7%,77.5%,NaN,0.2%,0.9%,3.7%


## Equity Geography

In [17]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 1]
        if len(df) > 0:
            df = calculate_mode_share(df, col)
            df['source'] = col
            result_df = pd.concat([df, result_df])
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_disability_reg,1.5%,21.3%,13.6%,44.0%,1.5%,0.4%,2.1%,15.5%
hh_poverty_50,2.5%,19.2%,12.8%,34.9%,1.4%,0.5%,6.1%,22.7%
hh_poverty_reg,1.7%,21.1%,13.9%,42.8%,1.6%,0.4%,2.6%,16.1%
hh_racial_50,1.6%,21.2%,14.4%,42.4%,1.7%,0.4%,2.7%,15.6%
hh_racial_reg,1.6%,21.2%,14.3%,42.8%,1.7%,0.4%,2.5%,15.5%
hh_english_reg,1.5%,21.8%,14.7%,44.2%,1.7%,0.3%,2.3%,13.5%
hh_elderly_50,0.5%,25.9%,10.3%,56.7%,0.7%,0.2%,0.3%,5.5%
hh_elderly_reg,1.3%,21.9%,13.9%,45.7%,1.5%,0.3%,2.0%,13.4%
hh_youth_reg,1.3%,23.0%,16.3%,45.3%,2.0%,0.2%,1.2%,10.6%


In [18]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 0]
        if len(df) > 0:
            df = calculate_mode_share(df, col)
            df['source'] = 'NOT in '+col
            result_df = pd.concat([result_df,df])
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
NOT in hh_youth_reg,1.7%,19.8%,12.0%,43.6%,1.2%,0.5%,3.1%,18.1%
NOT in hh_youth_50,1.5%,21.4%,14.1%,44.5%,1.6%,0.3%,2.2%,14.4%
NOT in hh_elderly_reg,1.6%,20.9%,14.3%,43.4%,1.7%,0.3%,2.3%,15.4%
NOT in hh_elderly_50,1.5%,21.4%,14.1%,44.4%,1.6%,0.3%,2.2%,14.4%
NOT in hh_english_reg,1.5%,21.1%,13.8%,44.6%,1.5%,0.3%,2.1%,15.0%
NOT in hh_english_50,1.5%,21.4%,14.1%,44.5%,1.6%,0.3%,2.2%,14.4%
NOT in hh_racial_reg,1.4%,21.5%,14.0%,45.9%,1.5%,0.3%,1.9%,13.5%
NOT in hh_racial_50,1.4%,21.5%,14.0%,45.4%,1.6%,0.3%,1.9%,13.9%
NOT in hh_poverty_reg,1.4%,21.6%,14.3%,45.6%,1.6%,0.3%,1.9%,13.4%
NOT in hh_poverty_50,1.5%,21.4%,14.2%,44.6%,1.6%,0.3%,2.1%,14.3%


In [19]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 1]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Non-Work')
            df['source'] = col
            result_df = pd.concat([result_df, df])
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_nonwork.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_youth_reg,1.3%,24.9%,17.7%,41.0%,2.3%,0.2%,1.2%,11.4%
hh_elderly_reg,1.4%,23.7%,15.0%,41.8%,1.7%,0.3%,2.0%,14.1%
hh_elderly_50,0.5%,26.9%,10.5%,55.3%,0.7%,0.2%,0.3%,5.5%
hh_english_reg,1.5%,23.6%,16.0%,39.9%,2.0%,0.3%,2.3%,14.3%
hh_racial_reg,1.6%,23.1%,15.6%,38.6%,1.9%,0.3%,2.5%,16.3%
hh_racial_50,1.7%,23.0%,15.7%,38.2%,1.9%,0.3%,2.8%,16.4%
hh_poverty_reg,1.7%,22.8%,15.0%,38.7%,1.8%,0.3%,2.6%,17.0%
hh_poverty_50,2.5%,20.6%,13.9%,31.1%,1.6%,0.5%,6.2%,23.6%
hh_disability_reg,1.5%,23.1%,14.8%,40.1%,1.7%,0.3%,2.1%,16.4%


In [20]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 0]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Non-Work')
            df['source'] = 'NOT in '+col
            result_df = pd.concat([result_df, df])
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_nonwork.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
NOT in hh_youth_reg,1.7%,21.6%,13.1%,39.7%,1.4%,0.4%,3.1%,19.0%
NOT in hh_youth_50,1.5%,23.2%,15.4%,40.3%,1.9%,0.3%,2.2%,15.3%
NOT in hh_elderly_reg,1.7%,22.7%,15.7%,39.0%,2.0%,0.3%,2.3%,16.3%
NOT in hh_elderly_50,1.5%,23.2%,15.4%,40.3%,1.9%,0.3%,2.2%,15.3%
NOT in hh_english_reg,1.5%,22.9%,15.0%,40.6%,1.8%,0.3%,2.0%,15.9%
NOT in hh_english_50,1.5%,23.2%,15.4%,40.3%,1.9%,0.3%,2.2%,15.3%
NOT in hh_racial_reg,1.4%,23.3%,15.2%,41.8%,1.8%,0.2%,1.8%,14.4%
NOT in hh_racial_50,1.4%,23.3%,15.2%,41.3%,1.8%,0.3%,1.9%,14.7%
NOT in hh_poverty_reg,1.4%,23.5%,15.6%,41.4%,1.9%,0.3%,1.9%,14.1%
NOT in hh_poverty_50,1.5%,23.3%,15.4%,40.5%,1.9%,0.3%,2.1%,15.1%


In [21]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 1]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Work')
            df['source'] = col
            result_df = pd.concat([result_df, df])
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_work.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_youth_reg,0.9%,10.2%,6.7%,75.6%,0.0%,0.3%,1.4%,4.9%
hh_elderly_reg,1.2%,9.5%,6.0%,72.7%,0.0%,0.5%,2.2%,7.9%
hh_elderly_50,0.3%,11.1%,6.2%,76.3%,NaN,0.2%,0.5%,5.3%
hh_english_reg,1.4%,9.8%,6.2%,72.1%,0.0%,0.5%,2.0%,8.0%
hh_racial_reg,1.6%,9.4%,5.9%,69.5%,0.0%,0.7%,2.2%,10.7%
hh_racial_50,1.6%,9.5%,6.0%,69.1%,0.0%,0.7%,2.3%,10.8%
hh_poverty_reg,1.6%,9.4%,5.9%,70.2%,0.0%,0.6%,2.6%,9.7%
hh_poverty_50,2.6%,8.0%,4.8%,62.9%,NaN,0.9%,5.4%,15.4%
hh_disability_reg,1.4%,9.3%,5.9%,71.0%,0.0%,0.6%,2.2%,9.5%


In [22]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 0]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Work')
            df['source'] = 'NOT in '+col
            result_df = pd.concat([result_df, df])
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_work.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))


Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
NOT in hh_youth_reg,1.8%,8.8%,5.2%,68.0%,0.0%,0.8%,3.0%,12.3%
NOT in hh_youth_50,1.4%,9.4%,5.9%,71.5%,0.0%,0.6%,2.3%,8.9%
NOT in hh_elderly_reg,1.6%,9.4%,5.8%,70.6%,0.0%,0.6%,2.3%,9.7%
NOT in hh_elderly_50,1.4%,9.4%,5.9%,71.5%,0.0%,0.6%,2.3%,8.9%
NOT in hh_english_reg,1.4%,9.2%,5.7%,71.2%,0.0%,0.6%,2.5%,9.5%
NOT in hh_english_50,1.4%,9.4%,5.9%,71.5%,0.0%,0.6%,2.3%,8.9%
NOT in hh_racial_reg,1.3%,9.4%,5.9%,73.4%,0.0%,0.4%,2.3%,7.3%
NOT in hh_racial_50,1.3%,9.4%,5.8%,72.7%,0.0%,0.5%,2.3%,8.0%
NOT in hh_poverty_reg,1.3%,9.4%,5.9%,72.4%,0.0%,0.5%,2.1%,8.4%
NOT in hh_poverty_50,1.4%,9.4%,5.9%,71.7%,0.0%,0.6%,2.2%,8.8%
